# Iris - sns.load_dataset('iris')

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
iris = sns.load_dataset('iris')

## a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width),
꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등
기초통계량(describe())을 구하시오.


In [3]:
iris.groupby('species').agg(['mean', 'std'])

sepal_length           sepal_width  ... petal_length petal_width          
                   mean       std        mean  ...          std        mean       std
species                                        ...                                   
setosa            5.006  0.352490       3.428  ...     0.173664       0.246  0.105386
versicolor        5.936  0.516171       2.770  ...     0.469911       1.326  0.197753
virginica         6.588  0.635880       2.974  ...     0.551895       2.026  0.274650

[3 rows x 8 columns]

## b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.
즉,
Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
s = iris[iris.species == 'setosa']['sepal_width']
s.mean(), s.std()

(3.428000000000001, 0.3790643690962886)

In [6]:
q1, q3 = s.quantile(.25), s.quantile(.75)
iqr = q3 - q1
q1, q3, iqr

(3.2, 3.6750000000000003, 0.4750000000000001)

In [7]:
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
s[(s < lower) | (s > upper)] = np.nan
s.isnull().sum()

2

In [8]:
s.mean(), s.std()

(3.4312500000000004, 0.32034306743094015)

In [12]:
def get_new_stat(s):
    q1, q3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    s[(s < lower) | (s > upper)] = np.nan
    outlier = s.isnull().sum() != 0
    return outlier, np.round(s.mean(), 2), np.round(s.std(), 4)

In [13]:
species_list, feature_list, mean_list, std_list = [],[],[],[]
outlier_list, new_mean_list, new_std_list = [],[],[]
for species in iris.species.unique():
    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        species_list.append(species)
        feature_list.append(feature)
        mean_list.append(np.round(s.mean(), 2))
        std_list.append(np.round(s.std(), 4))

        outlier, new_mean, new_std = get_new_stat(s)
        outlier_list.append(outlier)
        new_mean_list.append(new_mean)
        new_std_list.append(new_std)

In [14]:
df = pd.DataFrame({
    'species': species_list, 'feature': feature_list,
    'mean': mean_list, 'std': std_list,
    'outlier': outlier_list, 'new_mean': new_mean_list, 'new_std': new_std_list
})
df.set_index(['species','feature'], inplace=True)
df

mean     std  outlier  new_mean  new_std
species    feature                                               
setosa     sepal_length  5.01  0.3525    False      5.01   0.3525
           sepal_width   3.43  0.3791     True      3.43   0.3203
           petal_length  1.46  0.1737     True      1.46   0.1291
           petal_width   0.25  0.1054     True      0.23   0.0859
versicolor sepal_length  5.94  0.5162    False      5.94   0.5162
           sepal_width   2.77  0.3138    False      2.77   0.3138
           petal_length  4.26  0.4699     True      4.29   0.4378
           petal_width   1.33  0.1978    False      1.33   0.1978
virginica  sepal_length  6.59  0.6359     True      6.62   0.5935
           sepal_width   2.97  0.3225     True      2.96   0.2603
           petal_length  5.55  0.5519    False      5.55   0.5519
           petal_width   2.03  0.2747    False      2.03   0.2747